# ANPD Train

In [1]:
# Install Libraries
!pip install ultralytics==8.0.134 onnx onnxruntime --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.0 MB/s eta 0:00:00


In [5]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.3/78.2 GB disk)


In [4]:
import os

HOME = os.getcwd()
print(HOME)

/content


In [11]:
# Load Data
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet

from roboflow import Roboflow


rf = Roboflow(api_key="sOd6IHKIcXdroCfAXoWn")
project = rf.workspace("abdulazeez-jimoh-1qumc").project("sarkin-kofa-anpd")
dataset = project.version(1).download("yolov8")


/content/datasets
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.8 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.20, to fix: `pip install ultralytics==8.0.134`


Extracting Dataset Version Zip to anpr-1 in yolov8:: 100%|██████████| 514/514 [00:00<00:00, 2126.02it/s]


In [12]:
from ultralytics import YOLO
import shutil


In [14]:
# Nano Model
%cd {HOME}

yolov8n = YOLO('yolov8n.pt')
name = "anprv8n"

yolov8n.train(data=f"{dataset.location}/data.yaml", epochs=100, imgsz=640, name=name)

# Export the model
yolov8n.export(format='onnx')
shutil.copy2(f"runs/detect/{name}/weights/best.onnx", f"{name}.onnx")
shutil.copy2(f"runs/detect/{name}/weights/best.pt", f"{name}.pt")


Ultralytics YOLOv8.0.20 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)


/content


Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from runs/detect/anprv8n/weights/best.pt with output shape (1, 8, 8400) (6.0 MB)

ONNX: starting export with onnx 1.14.0...
ONNX: export success ✅ 0.7s, saved as runs/detect/anprv8n/weights/best.onnx (11.7 MB)

Export complete (1.0s)
Results saved to /content/runs/detect/anprv8n/weights
Predict:         yolo task=detect mode=predict model=runs/detect/anprv8n/weights/best.onnx -WARNING ⚠️ not yet supported for YOLOv8 exported models
Validate:        yolo task=detect mode=val model=runs/detect/anprv8n/weights/best.onnx -WARNING ⚠️ not yet supported for YOLOv8 exported models
Visualize:       https://netron.app


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



'anprv8n.pt'

In [15]:
# Small Model
%cd {HOME}

yolov8s = YOLO('yolov8s.pt')
name = "anprv8s"

yolov8s.train(data=f"{dataset.location}/data.yaml", epochs=100, imgsz=640, name=name)

# Export the model
yolov8s.export(format='onnx')
shutil.copy2(f"runs/detect/{name}/weights/best.onnx", f"{name}.onnx")
shutil.copy2(f"runs/detect/{name}/weights/best.pt", f"{name}.pt")


/content


100%|██████████| 21.5M/21.5M [00:00<00:00, 339MB/s]

Ultralytics YOLOv8.0.20 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/content/datasets/anpr-1/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=None, name=anprv8s, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=Fa

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



ONNX: export success ✅ 1.4s, saved as runs/detect/anprv8s/weights/best.onnx (42.7 MB)

Export complete (2.1s)
Results saved to /content/runs/detect/anprv8s/weights
Predict:         yolo task=detect mode=predict model=runs/detect/anprv8s/weights/best.onnx -WARNING ⚠️ not yet supported for YOLOv8 exported models
Validate:        yolo task=detect mode=val model=runs/detect/anprv8s/weights/best.onnx -WARNING ⚠️ not yet supported for YOLOv8 exported models
Visualize:       https://netron.app


'anprv8s.pt'

In [ ]:
# Medium Model

yolov8m = YOLO('yolov8m.pt')
name = "anprv8m"

yolov8m.train(data=f"{dataset.location}/data.yaml", epochs=100, imgsz=640, name=name)

# Export the model
yolov8m.export(format='onnx')
shutil.copy2(f"runs/detect/{name}/weights/best.onnx", f"{name}.onnx")
shutil.copy2(f"runs/detect/{name}/weights/best.pt", f"{name}.pt")


100%|██████████| 49.7M/49.7M [00:00<00:00, 126MB/s]
Ultralytics YOLOv8.0.145 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=data/anpr/anpr.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=anprv8s_100e, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript,

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



ONNX: export success ✅ 21.2s, saved as 'runs/detect/anprv8s_100e2/weights/best.onnx' (98.8 MB)

Export complete (25.7s)
Results saved to /content/drive/MyDrive/Colab Store/nigeria-anpr/runs/detect/anprv8s_100e2/weights
Predict:         yolo predict task=detect model=runs/detect/anprv8s_100e2/weights/best.onnx imgsz=640 
Validate:        yolo val task=detect model=runs/detect/anprv8s_100e2/weights/best.onnx imgsz=640 data=None 
Visualize:       https://netron.app


'anprv8s_100e.pt'